In [49]:
import pandas as pd
import numpy as np
import os

from collections import Counter
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
# from sklearn.metrics import accuracy_score,confusion_matrix, classification_report, balanced_accuracy_score
# from sklearn.metrics import auc,precision_recall_curve
# from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import f1_score, recall_score, precision_score, make_scorer
f1_scorer = make_scorer(f1_score, average="macro")

# from graphviz import Source
# from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier,AdaBoostClassifier,RandomForestClassifier,BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

# from imblearn.under_sampling import NearMiss
# from imblearn.over_sampling import SMOTE,ADASYN

# import matplotlib.pyplot as plt
# %matplotlib inline

In [3]:
def load_data(data_dir, window_size):
    '''
    data_dir (str): Base directory of data 
    window_size (str): Window size for input examples
    window_func (str): Window function reference as defined in utils.preprocess
                       Option are either 'window' or 'window_func'
    '''
    train_dir = os.path.join(data_dir, 'training')
    train_str = os.path.join(train_dir, 'training_{}.csv')
    test_str = os.path.join(data_dir, 'dataset_{}.csv')

    train_xs = []
    train_ys = []
    for i in [1, 2, 3, 4, 5, 100]: # file name updated to v2
        train_df_i = pd.read_csv(train_str.format(str(i)))
        
    # adding padded values and then windowing
        #         train_xi = window_func(train_df_i.kpi_value.values, window_size)
        local_min = train_df_i.kpi_value[0:window_size].min() # Using the global min as the padding 
        pad_min = np.ones(window_size) * local_min
        x_padded = np.concatenate([pad_min, train_df_i.kpi_value.values])
        train_xi = [x_padded[j:j+window_size] for j in range(len(x_padded)-(window_size))]
     
        train_xs.append(train_xi)
        train_ys.append(train_df_i.anomaly_label.values)
    x_train = np.concatenate(train_xs)
    y_train = np.concatenate(train_ys)
    assert len(x_train) == len(y_train)
    
    test_xs = []
    test_ys = []
    for i in [1,2,3,4,5,6,7,8,9,10,11,12,13,100,101,102,103,105,106]:  # file name updated to v2
        test_df_i = pd.read_csv(test_str.format(str(i)))
#         test_xi = window_func(test_df_i.values[:,1], window_size)
        test_local_min = test_df_i.kpi_value.values[0:window_size].min() # Using the local min as the padding 
        test_pad_min = np.ones(window_size) * test_local_min
        test_x_padded = np.concatenate([test_pad_min, train_df_i.kpi_value.values])
        test_xi = [test_x_padded[j:j+window_size] for j in range(len(test_x_padded)-(window_size))]

    test_xs.append(test_xi)
    x_test = np.concatenate(test_xs)
    print("Train_x shape: {}\nTrain_y shape: {}\n\nTest_x shape: {}".format(x_train.shape, y_train.shape, x_test.shape))
    return x_train, y_train, x_test

def window_min_max(x):
    x_min = x.min(axis=1).reshape(-1, 1)
    x_max = x.max(axis=1).reshape(-1, 1)
    for i in range(len(x)):
        if x_max[i] > x_min[i]:
            x[i] =  (x[i] - x_min[i])/(x_max[i] - x_min[i])
        else:  # add scenario where x_max = x_min in a window 
            x[i] = 0
    return x

In [7]:
# os.getcwd()

'C:\\Users\\Pheobe\\Documents\\[B]Huawei-Challenge-2020\\draft'

In [10]:
data_dir_v2 ="../for_students/data_v2"
x, y, z = load_data(data_dir_v2, 100) # z is the target_x

Train_x shape: (54337, 100)
Train_y shape: (54337,)

Test_x shape: (20159, 100)


In [11]:
# Window level normalisation
x = window_min_max(x)

In [14]:
# Cross val incides
skf = StratifiedKFold(n_splits=5)

In [15]:
# Alternatively using Hold-out testing
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.4, random_state=42)

# Results

In [45]:
# KNN_3
kNN_3 = KNeighborsClassifier(n_neighbors=3)
kNN_3_scores = cross_val_score(kNN_3, x, y, scoring='f1_macro', cv=skf, n_jobs=-1, error_score='raise')
for score in list(kNN_3_scores):
    print("KNN3 - f1-score: {:.2f}".format(score))

KNN3 - f1-score: 0.58
KNN3 - f1-score: 0.85
KNN3 - f1-score: 0.68
KNN3 - f1-score: 0.63
KNN3 - f1-score: 0.66


In [46]:
# KNN_5
kNN_5 = KNeighborsClassifier(n_neighbors=5)
kNN_5_scores = cross_val_score(kNN_5, x, y, scoring='f1_macro', cv=skf, n_jobs=-1, error_score='raise')
for score in list(kNN_5_scores):
    print("KNN5 - f1-score: {:.2f}".format(score))

KNN5 - f1-score: 0.60
KNN5 - f1-score: 0.83
KNN5 - f1-score: 0.65
KNN5 - f1-score: 0.63
KNN5 - f1-score: 0.67


In [47]:
# Logistic Regression
logistic_reg = LogisticRegression(max_iter=4000)
lr_scores = cross_val_score(logistic_reg, x, y, scoring='f1_macro', cv=skf, n_jobs=-1, error_score='raise')
for score in list(lr_scores):
    print("Logistic Regression - f1-score: {:.2f}".format(score))

Logistic Regression - f1-score: 0.49
Logistic Regression - f1-score: 0.49
Logistic Regression - f1-score: 0.49
Logistic Regression - f1-score: 0.49
Logistic Regression - f1-score: 0.49


In [48]:
# Gradient Boosting
grad_boost = GradientBoostingClassifier(max_depth=5)
gb_scores = cross_val_score(grad_boost, x, y, scoring='f1_macro', cv=skf, n_jobs=-1, error_score='raise')
for score in list(gb_scores):
    print("Gradient Boosting - f1-score: {:.2f}".format(score))

Gradient Boosting - f1-score: 0.64
Gradient Boosting - f1-score: 0.76
Gradient Boosting - f1-score: 0.58
Gradient Boosting - f1-score: 0.62
Gradient Boosting - f1-score: 0.52


In [50]:
# Naive Bayes
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
gnb = GaussianNB()
bnb = BernoulliNB()
mnb = MultinomialNB()
y_pred = gnb.fit(x_train,y_train).predict(x_test)

gnb_scores = cross_val_score(gnb, x, y, scoring='f1_macro', cv=skf, n_jobs=-1, error_score='raise')
for score in list(gnb_scores):
    print("Gaussian Naive Bayes - f1-score: {:.2f}".format(score))

bnb_scores = cross_val_score(bnb, x, y, scoring='f1_macro', cv=skf, n_jobs=-1, error_score='raise')
for score in list(bnb_scores):
    print("Bernoulli Naive Bayes - f1-score: {:.2f}".format(score))

Gaussian Naive Bayes - f1-score: 0.30
Gaussian Naive Bayes - f1-score: 0.50
Gaussian Naive Bayes - f1-score: 0.15
Gaussian Naive Bayes - f1-score: 0.14
Gaussian Naive Bayes - f1-score: 0.04
Bernoulli Naive Bayes - f1-score: 0.27
Bernoulli Naive Bayes - f1-score: 0.40
Bernoulli Naive Bayes - f1-score: 0.49
Bernoulli Naive Bayes - f1-score: 0.49
Bernoulli Naive Bayes - f1-score: 0.49


In [51]:
# Random Forest
random_forest = RandomForestClassifier()
rf_scores = cross_val_score(random_forest, x, y, scoring='f1_macro', cv=skf, n_jobs=-1, error_score='raise')
for score in list(rf_scores):
    print("Random Forest - f1-score: {:.2f}".format(score))

Random Forest - f1-score: 0.63
Random Forest - f1-score: 0.63
Random Forest - f1-score: 0.49
Random Forest - f1-score: 0.60
Random Forest - f1-score: 0.49


In [52]:
# SVM
svc = LinearSVC()
svc_scores = cross_val_score(svc, x, y, scoring='f1_macro', cv=skf, n_jobs=-1, error_score='raise')
for score in list(svc_scores):
    print("SVM - f1-score: {:.2f}".format(score))

SVM - f1-score: 0.49
SVM - f1-score: 0.49
SVM - f1-score: 0.49
SVM - f1-score: 0.49
SVM - f1-score: 0.49
